In [67]:
# Import libraries

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,learning_curve, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

from sklearn.linear_model import LassoCV,ElasticNetCV,Lasso,ElasticNet
from sklearn.kernel_ridge import KernelRidge


# 아래 추가적인 패키지가 설치가 되어 있어야 함!!!!!!!
from xgboost import XGBRegressor
from mlxtend.regressor import StackingRegressor


# 시간
import time

##### 평가 및 kfold 사용 준비

In [45]:
kfold = KFold(n_splits=6, random_state= 0, shuffle = True)

In [46]:
def rmsle_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_df, 
                                   scoring="neg_mean_squared_error", cv = kfold))
    return(rmse)

##### Rigid 

In [ ]:
from sklearn.kernel_ridge import KernelRidge

* ref ) https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html#sklearn.kernel_ridge.KernelRidge

In [ ]:
KR = KernelRidge()
# 기본적으로 해당하지 않는 설절들에 대해서는 KernelRidge에서 알아서 패스하고 안 하는 부분이 있어서 에러에 크게 염려하지 않아도 도 됨!!!
KR_param_grid = {
    # alpha : 기보적인 모든 모델에 대한 variance를 줄이기 위한 값들.. 기본 1
    # 너무 오래 걸려서 좀 간단히 줄여서...
    #'alpha' : [0.0001, 0.0005, 0.0008, 0.001, 0.005, 0.009, 0.01, 0.03, 0.05, 0.07, 0.09, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 2, 3, 4],
    'alpha' : [0.01, 0.1, 0.5, 1, 1.5, 2 ],
    # 사용할 커널의 종류들 : 기본은 linear  이고, 여러가지가 존재함!!!
    # 'kernel' : ['polynomial',"linear","rbf"],
    'kernel' : ['polynomial',"rbf"],
    # 위의 커널에서  RBF, laplacian, polynomial, exponential chi2 and sigmoid kernels 등을 사용할 때 자동 선택된다.
    'gamma':np.logspace(-15, 4, num = 5, base = 2.0),
    # 커널에서 Poly에 대한 것을 설정할 때 사용된다!!!
    'degree': [2,3],
    # 커널 : polynomial and sigmoid kernels 을 사용할 때, 상수항에 대한 계수...
    'coef0': [0.5, 1.0, 1.5, 2.0]
}

t1 = time.time()
KR_CV = GridSearchCV(KR, 
                     param_grid = KR_param_grid, 
                     cv = kfold, 
                     scoring = "neg_mean_squared_error",
                     n_jobs = -1, 
                     verbose = 1)
KR_CV.fit(X_train, y_df)
t2 = time.time()
print("Run Time : ", str(t2-t1))
KR_best = KR_CV.best_estimator_
print(KR_best)

In [47]:
score = rmsle_cv(KR_best)
print("Kernel Ridge mean score:", score.mean())
print("Kernel Ridge std:", score.std())

Kernel Ridge mean score: 0.11196564169797985
Kernel Ridge std: 0.007929655583876188


* 위의 부분을 너무 많이 설정을 하게 되면, 좀 자세히 찾을 수 있겠지만, 너무 시간이 오래 걸리는 부분들이 발생을 한다. 그래서 아래와 같이 randomSearchCV를 사용해보려고 함!!!

In [50]:
score = rmsle_cv(KR_best_RS)
print("Kernel Ridge mean score:", score.mean())
print("Kernel Ridge std:", score.std())

Kernel Ridge mean score: 0.11220443541311469
Kernel Ridge std: 0.007863732026258856


In [49]:
#
t1 = time.time()
n_iter_search = 20
random_search = RandomizedSearchCV(KR, param_distributions=KR_param_grid,
                                   n_iter=n_iter_search, cv=kfold, scoring="neg_mean_squared_error")

start = time.time()
KR_RS = random_search.fit(X_train, y_df)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))

KR_best_RS = KR_RS.best_estimator_
print(KR_best_RS)

RandomizedSearchCV took 25.58 seconds for 20 candidates parameter settings.
KernelRidge(alpha=1.5, coef0=2.0, degree=3, gamma=0.0008211879055212056,
            kernel='polynomial', kernel_params=None)


In [52]:
# 참고 ) np.explm1 : exp(x) - 1 ---> 다시 복기하는 부분..
y_submission_1 = np.expm1(KR_best_RS.predict(X_test))

##### Lasso Regression

In [ ]:
from sklearn.linear_model import LassoCV,ElasticNetCV,Lasso,ElasticNet

* ref) https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [53]:
Las = Lasso()

Las_param_grid = {
    # alpha : 기보적인 모든 모델에 대한 variance를 줄이기 위한 값들.. 기본 1, 0일 때는 기존의 ord와 동일!!!
    'alpha' : [0.0001, 0.0005, 0.0008, 0.001, 0.005, 0.009, 0.01, 0.03, 0.05, 0.07, 0.09, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 2, 3, 4],
    # 모델을 할 때 상수항을 할지 말지 : 기본은 안 한다
    'fit_intercept' : [True, False],
    # 정규화에 대한 부분 : True를 하면 자동으로 정규화를 L2로 수행을 하고, 아니면 그 전에 StanderScaler 사용
    'normalize' : [False, True],
}

t1 = time.time()
Las_CV = GridSearchCV(Las, 
                     param_grid = Las_param_grid, 
                     cv = kfold, 
                     scoring = "neg_mean_squared_error",
                     n_jobs = -1, 
                     verbose = 1)
Las_CV.fit(X_train, y_df)
t2= time.time()
print("Run Time :", str(t2-t1))
Las_best = Las_CV.best_estimator_
print(Las_best)

Fitting 6 folds for each of 40 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.4s


Run Time : 9.697360038757324
Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)


[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    9.6s finished


In [54]:
print("Lasso mean score:", rmsle_cv(Las_best).mean())
print("Lasso std:", rmsle_cv(Las_best).std())


Lasso mean score: 0.11250935655377242
Lasso std: 0.00903330132980516


##### ElasticNet Regression

In [ ]:
from sklearn.linear_model import LassoCV,ElasticNetCV,Lasso,ElasticNet

In [58]:
elnet = ElasticNet()

elnet_param_grid = {
    # 기존 : 위에서 한 alpha의 기능 동일!!!
    'alpha' : [0.01, 0.1, 0.5, 1, 1.5, 2 ],
    # ElasticNet mixing parameter, with 0 <= l1_ratio <= 1
    # l1_ratio = 0 the penalty is an L2 penalty. For l1_ratio = 1 it is an L1 penalty.
    'l1_ratio' : [0.01, 0.1, 0.2, 0.5, 0.7, 0.8],
    # 기존 : 모델을 할 때 상수항을 할지 말지 : 기본은 안 한다
    'fit_intercept': [True, False], 
    # 기존 : 정규화에 대한 부분 : True를 하면 자동으로 정규화를 L2로 수행을 하고, 아니면 그 전에 StanderScaler 사용
    'normalize' : [False, True],
    # Whether to use a precomputed Gram matrix to speed up calculations
    "precompute" : [True, False],
    # The maximum number of iterations
    "max_iter" : [1000, 3000],
    # The tolerance for the optimization: if the updates are smaller than tol, the optimization code checks the dual gap for optimality and continues until it is smaller than tol.
    "tol" : [0.0001, 0.00001]
}



t1 = time.time()
elnet_GS = GridSearchCV(elnet, 
                     param_grid = elnet_param_grid, 
                     cv = kfold, 
                     scoring = "neg_mean_squared_error",
                     n_jobs = -1, 
                     verbose = 1)
elnet_GS.fit(X_train, y_df)
t2= time.time()
print("Run Time :", str(t2-t1))
elnet_GS_best = elnet_GS.best_estimator_
print(elnet_GS_best)

Fitting 6 folds for each of 576 candidates, totalling 3456 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   41.4s
[Parallel(n_jobs=-1)]: Done 1232 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 3232 tasks      | elapsed:  1.7min


Run Time : 101.20419096946716
ElasticNet(alpha=0.01, copy_X=True, fit_intercept=True, l1_ratio=0.2,
           max_iter=1000, normalize=False, positive=False, precompute=True,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)


[Parallel(n_jobs=-1)]: Done 3456 out of 3456 | elapsed:  1.7min finished


In [59]:
print("ElasticNet mean score:", rmsle_cv(elnet_GS_best).mean())
print("ElasticNet std:", rmsle_cv(elnet_GS_best).std())

ElasticNet mean score: 0.11166291672292976
ElasticNet std: 0.009169557729243012


In [60]:
y_submission_3 = np.expm1(elnet_GS_best.predict(X_test))

##### XG Boost¶

* ref) https://xgboost.readthedocs.io/en/latest/parameter.html#general-parameters   

* 파라미터 : 일반 파라미터 / 부스트 파라미터 / 학습과정 파라미터/ 커맨드 라인 파리미터     
** 일반 파라미터   
1) booster : 어떤 부스팅 방식을 사용할 지 결정 - gbtree, gblinear, dart etc   
2) nthread : 몇 개의 thread를 사용해서 동시에 처리할지 - 기본은 최대한 많이 --> 특별히 선택하지 않아도 됨!   
3) num_feature  : 특징의 차원의 숫자를 정해야 하는 경우 옵션을 통해서 처리 - 기본은 최대한 많이 이기에, 특별하게 선택하지 않아도 될 듯   
** 부스팅 파라미터   
1) eta : 러닝 레이트임. 트리에 가지가 많을 수록 overfitting이 발생하기에, 매번 부스팅 스탭마다 weight를 주어서 부스팅 과정에서 과접합이 일어나지 않도록 함.   
2) gamma : 정보 획득에서 -r로 펴현이 되는 부분이 있는 감마임. 이 값이 커지게 되면 ㅡㅌ리의 깊이가 줄어들어서 좀 더 보수적인 모델이 된다. 기본값은 0임.  
3) max_depth : 한 트리의 최대 깊이를 지정하는 것. 숫자가 커지면 모델의 복잡도가 올라가고, 당연히 overfitting의 위험이 발생하게 된다. 기본은 6으로 되어 있어서 최대는 2^6=64개가 된다.  
4) lambda(L2 reg-form) : L2 정규화에 달리는 weight임. 숫자가 클 수록 보수적인 모델임.
5) alpha(L1 reg-form) : L1 정규화에 달리는 weight임. 숫자가 클 수록 보수적인 모델이 됨. 그리고 L2보다 아웃라이어에 좀 더 민감하게 작용을 하는 특징이 있음!!!   
** 학습 과정 파라미터   
1) objective : 목적 함수 (reg:linear, binary:logistic, count:possion  etc)   
2) eval_metric : 모델의 평가 함수를 조정하는 함수 . rmse( root mean square error). logloss(log-likelihood), map(mean average precision) etc   
** 커맨드 라인 파라미터   
1) num_rounds : boosting 라운드를 결정한다. 랜덤하게 하게 되니 적당히 크게 되어야 함. epoch와 동일.


In [73]:
XGB = XGBRegressor(objective ='reg:squarederror')
#XGB = XGBRegressor()

xg_param_grid = {
              'n_estimators' :[870],
              'learning_rate': [0.04],
              
              'max_depth': [3],
              'min_child_weight':[0.2],
              
              'gamma': [0],
                
              'subsample':[0.8],
              'colsample_bytree':[0.7]
    
              #'reg_alpha':[0.08,0.09,0.095,0.1,0.15,0.2],
              #'reg_lambda':[0,0.001,0.002]
              }
                
gsXGB = GridSearchCV(XGB,param_grid = xg_param_grid, cv=kfold, scoring="neg_mean_squared_error", n_jobs= -1, verbose = 1)
gsXGB.fit(X_train,y_df)
XGB_best = gsXGB.best_estimator_
print(gsXGB.best_params_)

Fitting 6 folds for each of 1 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   35.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   35.3s finished


{'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.04, 'max_depth': 3, 'min_child_weight': 0.2, 'n_estimators': 870, 'subsample': 0.8}


In [79]:
print("XG Boost mean score:", rmsle_cv(XGB_best).mean())
print("XG Boost std:", rmsle_cv(XGB_best).std())


XG Boost mean score: 0.1136928510161249
XG Boost std: 0.00882904786198643


In [80]:
y_submission_6 = np.expm1(gsXGB.predict(X_test))

### Ensemble - Stacked Regression and GridSearch

* ref) http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/

In [62]:
print("source: https://rasbt.github.io/mlxtend/user_guide/regressor/StackingCVRegressor/")
Image(url= "https://rasbt.github.io/mlxtend/user_guide/regressor/StackingCVRegressor_files/stacking_cv_regressor_overview.png")

source: https://rasbt.github.io/mlxtend/user_guide/regressor/StackingCVRegressor/


In [69]:
XGB = XGBRegressor()

ELNET = ElasticNet(random_state = 1)
LCV=Lasso(random_state = 1)
SV = SVR()
KR = KernelRidge()
XG = XGBRegressor()
stack = StackingRegressor(regressors = [ELNET,LCV,XG,SV,KR], meta_regressor = XGB)

params = {  
    # mlxtend : 0.17 이하 버전기준..
    
#               'meta-xgbregressor__n_estimators' : [740*2],#740
#               'meta-xgbregressor__learning_rate': [0.01/2], #0.01
#                'meta-xgbregressor__min_child_weight':[0],
#               'meta-xgbregressor__gamma':[0.1],
#               'meta-xgbregressor__max_depth': [2],
#               'meta-xgbregressor__subsample':[0.65],
#               'meta-xgbregressor__colsample_bytree':[0.4],
#               'meta-xgbregressor__reg_alpha':[0],
#               'meta-xgbregressor__reg_lambda':[1],
    
        # mlxtend : 0.17 버전 기준 : http://rasbt.github.io/mlxtend/user_guide/regressor/StackingRegressor/
                'meta_regressor__n_estimators' : [740*2],#740
              'meta_regressor__learning_rate': [0.01/2], #0.01
               'meta_regressor__min_child_weight':[0],
              'meta_regressor__gamma':[0.1],
              'meta_regressor__max_depth': [2],
              'meta_regressor__subsample':[0.65],
              'meta_regressor__colsample_bytree':[0.4],
              'meta_regressor__reg_alpha':[0],
              'meta_regressor__reg_lambda':[1],

              
              'lasso__alpha':[0.00244736842105],
              'elasticnet__alpha':[0.0276315789474],
              'elasticnet__l1_ratio':[0.09],
              'xgbregressor__min_child_weight':[0.2],
              'xgbregressor__n_estimators' : [870],
              'xgbregressor__learning_rate': [0.04],
              'xgbregressor__gamma':[0],
              'xgbregressor__max_depth': [3],
              'xgbregressor__subsample':[0.8],
              'xgbregressor__colsample_bytree':[0.7],
    
              'kernelridge__alpha':[0.93],
              'kernelridge__coef0':[1.5],
              'kernelridge__degree':[3],
              'kernelridge__gamma':[0.001],
              'kernelridge__kernel':['polynomial'],
              'kernelridge__kernel_params':[None],
              
              'svr__coef0':[1.6],
              'svr__kernel':['poly'],
              'svr__epsilon':[0.03],
              'svr__gamma': ['auto'],
              'svr__degree': [2],
              'svr__C':[0.1]
        }

grid = GridSearchCV(estimator = stack, param_grid=params,cv=kfold,refit=True, verbose=1,n_jobs=-1,scoring="neg_mean_squared_error")
grid.fit(X_train, y_df)
grid_best = grid.best_estimator_
print(grid_best)


Fitting 6 folds for each of 1 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   45.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   45.0s finished


[09:43:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:43:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
StackingRegressor(meta_regressor=XGBRegressor(base_score=0.5, booster='gbtree',
                                              colsample_bylevel=1,
                                              colsample_bynode=1,
                                              colsample_bytree=1, gamma=0,
                                              importance_type='gain',
                                              learning_rate=0.1,
                                              max_delta_step=0, max_depth=3,
                                              min_child_weight=1, missing=None,
                                              n_estimators=100, n_jobs=1,
                                              nthread=None,
                                              objectiv

In [70]:
print("Stacking mean score:", rmsle_cv(grid_best).mean())
print("Stacking std:", rmsle_cv(grid_best).std())

[09:43:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:44:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [71]:
y_submission_st = np.expm1(grid.predict(X_test))

#### Ensemble - Averaging

In [88]:
y_submission_avg = (y_submission_6 + y_submission_2 + y_submission_st)/3

#y_submission_avg = ( 5 * y_submission_6 + 2 * y_submission_2 + 1 * y_submission_st)/8

In [90]:
y_submission_avg

array([124050.41552813, 162062.44566583, 190340.67219535, ...,
       163178.33501296, 117098.4432756 , 222331.02387479])

## 최종 제출

In [91]:
my_submission = pd.DataFrame()
my_submission['Id'] = test_id
my_submission['SalePrice'] = y_submission_avg
my_submission.to_csv('submission_trail_ver_0-1.csv',index=False)
